<a href="https://colab.research.google.com/github/takatakamanbou/MVA/blob/main/MVA2024_ex10notebookA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MVA2024 ex10notebookA

<img width=64 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/MVA-logo09.png"> https://www-tlab.math.ryukoku.ac.jp/wiki/?MVA/2024

---
## 多次元（多変量）正規分布 (2)
---



In [ ]:
# いつものいろいろインポート
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn
seaborn.set()

In [ ]:
# 追加でいろいろ

# NumPy の 疑似乱数生成器（rng = random number generator）
from numpy.random import default_rng
rng = default_rng() # 疑似乱数生成器を初期化

# SciPy の 統計関数群の中の正規分布モジュール (scipy.stats.norm) と多変量正規分布モジュール (scipy.stats.multivariate_normal)
from scipy.stats import norm, multivariate_normal

# 身長体重のデータ
dfHW = pd.read_csv('https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/hw.csv', header=0)

---
### 正規分布に従う乱数の生成

正規分布の性質を学ぶためには，実際に正規分布に従うデータを集めてきて観察してみるのが最も適切かもしれません．しかし，実データの場合，母平均や母分散（母集団の平均や分散）がいくつなのかわからないだけでなく，真に正規分布に従っているのかどうかも不明確なのが普通です．そのようなデータを使って学ぶのは難しいですので，ここではコンピュータに生成させた（ある意味きれいな）乱数データを使ってみましょう．

厳密なことを言うと，コンピュータによる計算で得られるのは，真の意味での乱数ではなく，「疑似乱数」です．しかし，実用上は乱数とみなせるような疑似乱数を得る方法がいろいろ知られています（注）．

<span style="font-size: 75%">
※ 注意: 例えば，C言語では rand 関数で整数の乱数値を得ることができます（が，あまりよい疑似乱数ではないので注意が必要．気になるひとは takataka に聞いたり調べたりしてね）．
</span>


#### 1次元正規分布に従う乱数



まずは，1次元標準正規分布に従う乱数を作ってみます．どのような計算でそのような乱数を作るのか，というのも面白い話なのですが，ここでは省略します（注）．


次のコードセルを実行すると，1次元標準正規分布に従う乱数値が20個表示されます．
実行するたびに新しい乱数値が生成されますので，何度か実行してみましょう．

<br>
<hr width="50%" align="left">
<span style="font-size: 75%">
※注： 一様分布する乱数値を変換して作ったりします．気になるひとは「ボックス=ミュラー法」とか調べてみるとよいでしょう．
</span>

In [ ]:
# 1次元標準正規分布に従う乱数値を 20 個取得
samples = rng.standard_normal(20)
print(samples)

コンピュータが生成する疑似乱数は，一定の規則に従った計算を繰り返して得られる数列です．ですので，その初期値（初項）が同じならば，常に同じ値が生成されます．
疑似乱数を生成する際に初期値を決めるのに使われる数のことを，乱数の種（たね，seed）といいます．

次のコードセルを実行すると，種を `5963` として乱数生成器を初期化しなおします．このセルを実行してから上のセルを実行すると，毎回同じ値が得られることを確認しましょう．種の値を変えれば，得られる値も変化します．

In [ ]:
rng = default_rng(5963) # 乱数の種を 5953 にして疑似乱数生成器を初期化

得られた値が標準正規分布に従っていることを直感的に確かめるため，ヒストグラムを描いてみましょう．

In [ ]:
# 1次元標準正規分布に従う乱数値を 200 個取得
samples = rng.standard_normal(100)

# それらのヒストグラムを標準正規分布の確率密度関数に重ねて描く
fig, ax = plt.subplots(figsize=(10, 5))
mu, sigma = 0.0, 1.0
xmin, xmax = mu - 5*sigma, mu + 5*sigma
x = np.linspace(xmin, xmax, 200)
bins = np.linspace(xmin, xmax, 100)
fx = norm.pdf(x, loc=mu, scale=sigma)
ax.hist(samples, bins=bins, density=True, label='samples')
ax.plot(x, fx, label=f'N({mu}, {sigma*sigma})', color='red', lw=3)
ax.axvline(0, color='gray')
ax.set_xlim(np.min(x), np.max(x))
ax.legend()
plt.show()

標準正規分布に従う乱数値が得られたら，簡単な計算によって，任意の平均と分散を持つ正規分布に従う乱数値を得ることができます．

$X$ が $N(0, 1)$ に従うとき，$Y = aX + b$ は平均 $b$ 分散 $a^2$ の正規分布すなわち $N(b, a^2)$ に従います．したがって，平均 $\mu$ 分散 $\sigma^2$ の正規分布に従う乱数値が欲しければ，次のようにすればよいことが分かります．
1. 標準正規分布に従う乱数を生成する
1. 得られた値を $\sigma$ 倍して $\mu$ を加えた値を計算する

以下，「正規分布に従う乱数」を省略して，「正規乱数」と呼ぶことがあります．

次のセルでは，平均 `mu` 標準偏差 `sigma` の正規乱数を200個作って，そのヒストグラムを描きます．`mu` や `sigma` をいろいろ変えて実行してみましょう．

In [ ]:
# 1次元標準正規分布に従う乱数値を 200 個取得
samples = rng.standard_normal(200)

# それを N(mu, sigma^2) に従う乱数値に変換
mu = 4.0
sigma = 3.0
samples2 = sigma*samples + mu

# それらのヒストグラムを N(mu, sigma^2) の確率密度関数に重ねて描く
fig, ax = plt.subplots(figsize=(10, 5))
xmin, xmax = mu - 5*sigma, mu + 5*sigma
x = np.linspace(xmin, xmax, 200)
bins = np.linspace(xmin, xmax, 100)
fx = norm.pdf(x, loc=mu, scale=sigma)
ax.hist(samples2, bins=bins, density=True, label='samples')
ax.plot(x, fx, label=f'N({mu}, {sigma*sigma})', color='red', lw=3)
ax.axvline(0, color='gray')
ax.set_xlim(np.min(x), np.max(x))
ax.legend()
plt.show()

#### 多次元正規分布に従う乱数

ここでは，2次元の場合に限って，正規分布に従う乱数を生成する場合について説明します．
ですが，3次元以上の場合についても同様に考えることができます．

$\pmb{X} = \begin{pmatrix} X \\ Y \end{pmatrix}$ が2次元の標準正規分布 $N(\pmb{0}, I)$ すなわち平均がゼロベクトル $\pmb{0} = \begin{pmatrix} 0 \\ 0\end{pmatrix}$ で分散共分散行列が単位行列 $I = \begin{pmatrix} 1 & 0 \\ 0 & 1 \end{pmatrix}$ の2次元正規分布に従う場合，$X$ と $Y$ は独立でかつそれぞれが $N(0, 1)$ に従うのでした．
逆に，互いに独立でかつそれぞれが $N(0, 1)$ に従う $X, Y$ があったとき，$\pmb{X} = \begin{pmatrix} X \\ Y \end{pmatrix}$ は $N(\pmb{0}, I)$ に従います．

したがって，標準正規分布に従う乱数値を2セット，独立に生成してやれば，2次元標準正規分布に従う乱数値を得ることができます．

In [ ]:
# 2次元標準正規分布に従う乱数値を 20 個取得
samples = rng.standard_normal((20, 2))
print(samples, samples.shape)

次のセルを実行すると，2次元標準正規分布に従う乱数値 500 個を散布図に描きます．この図には，2次元標準正規分布の確率密度関数の値が一定値（$0.1, 0.01, 0.001$）をとる点がつくる曲線（円になります）も描いてあります．

In [ ]:
# 2次元標準正規分布に従う乱数値を 500 個取得
samples = rng.standard_normal((500, 2))

# 確率密度描画のためのグリッドデータの作成
xmin, xmax = -5, 5
mu = np.zeros(2)
cov = np.diag(np.ones(2))
xx, yy = np.mgrid[xmin:xmax:0.01, xmin:xmax:0.01]
zz = multivariate_normal.pdf(np.dstack((xx, yy)), mean=mu, cov=cov)

# それらの散布図を標準正規分布の確率密度関数に重ねて描く
fig, ax = plt.subplots(figsize=(5, 5))
ax.scatter(samples[:, 0], samples[:, 1], label='samples', s=8)
ax.contour(xx, yy, zz, colors=['#ffa0a0', '#ff5050', '#ff0000'], levels=[0.001, 0.01, 0.1])
ax.set_xlim(xmin, xmax)
ax.axvline(0, color='gray')
ax.axhline(0, color='gray')
ax.set_xlim(xmin, xmax)
ax.set_ylim(xmin, xmax)
ax.set_aspect('equal')
ax.legend()
plt.show()

同様に，$N(\mu_x, \sigma_y^2)$ および $N(\mu_x, \sigma_y^2)$ に従う乱数値を独立に生成させることで，平均 $\pmb{\mu} = \begin{pmatrix} \mu_x \\ \mu_y \end{pmatrix}$，分散共分散行列 $\Sigma = \begin{pmatrix} \sigma_x^2 & 0 \\ 0 & \sigma_y^2 \end{pmatrix}$ の2次元正規分布 $N(\pmb{\mu}, \Sigma)$ に従う乱数値を生成することができます．

In [ ]:
# 2次元標準正規分布に従う乱数値を 500 個取得
samples = rng.standard_normal((500, 2))

# 平均と分散共分散行列
mu_x,   mu_y   = -2, 2
sig2_x, sig2_y = 1.5, 0.5
mu = np.array([mu_x, mu_y])
cov = np.diag(np.array([sig2_x, sig2_y]))

# 平均が mu_x, mu_y で分散が sig2_x, sig2_y になるように変換
samples[:, 0] = np.sqrt(sig2_x)*samples[:, 0] + mu_x
samples[:, 1] = np.sqrt(sig2_y)*samples[:, 1] + mu_y

# 実は上記のように自分で変換しなくても，以下のコードで平均 mu, 共分散 cov の正規乱数を生成できる
#samples = rng.multivariate_normal(mu, cov, size=100)

# 確率密度描画のためのグリッドデータの作成
xmin, xmax = -5, 5
xx, yy = np.mgrid[xmin:xmax:0.01, xmin:xmax:0.01]
zz = multivariate_normal.pdf(np.dstack((xx, yy)), mean=mu, cov=cov)

# それらの散布図を標準正規分布の確率密度関数に重ねて描く
fig, ax = plt.subplots(figsize=(5, 5))
ax.scatter(samples[:, 0], samples[:, 1], label='samples', s=8)
ax.contour(xx, yy, zz, colors=['#ffa0a0', '#ff5050', '#ff0000'], levels=[0.001, 0.01, 0.1])
ax.set_xlim(xmin, xmax)
ax.axvline(0, color='gray')
ax.axhline(0, color='gray')
ax.set_xlim(xmin, xmax)
ax.set_ylim(xmin, xmax)
ax.set_aspect('equal')
ax.legend()
plt.show()

この図には，確率密度関数の値が一定値（$0.1, 0.01, 0.001$）をとる点がつくる曲線（楕円になります）も描いてあります．

さて，それでは，平均 $\pmb{\mu}$ および分散共分散行列 $\Sigma$ として任意の値をもつ正規乱数を得るにはどうしたらよいでしょうか．
$N(\pmb{0}, \Sigma)$ に従う正規乱数が得られれば，それらを $\mu$ だけ平行移動すれば（得られた乱数値に $\pmb{\mu}$ を加えれば）$N(\pmb{\mu}, \Sigma)$ に従う正規乱数が得られます．
ですので，ここでは，$N(\pmb{0}, \Sigma)$ に従う正規乱数を得る方法を考えます．

ある分散共分散行列 $\pmb{\Sigma}$ の固有値とそれらに対応する単位固有ベクトルを $\lambda_1, \lambda_2$ および $\pmb{u}_1, \pmb{u}_2$ として，$2\times 2$ 行列 $U$ を $U = \begin{pmatrix} \pmb{u}_1 & \pmb{u}_2 \end{pmatrix}$ とおきます．
このとき，$U^{\top}\Sigma U = \begin{pmatrix} \lambda_1 & 0 \\ 0 & \lambda_2 \end{pmatrix}$ と対角化できる（注1）ことから，$\Sigma$ は次のように表せます．

$$
\Sigma = U\begin{pmatrix} \lambda_1 & 0 \\ 0 & \lambda_2 \end{pmatrix}U^{\top}
$$

ここで，2次元確率変数 $\pmb{Y}$ が平均 $\pmb{0}$ で分散共分散行列 $\begin{pmatrix}\lambda_1 & 0 \\ 0 & \lambda_2\end{pmatrix}$ の2次元正規分布に従うとして，$\pmb{X} = U\pmb{Y}$ という変数変換を考えてみます．
$\pmb{Y}$ の平均が $\pmb{0}$ である，すなわち $E[\pmb{Y}] = \pmb{0}$ である（注2）ことから，$\pmb{X}$ の平均も

$$
E[\pmb{X}] = E[U\pmb{Y}] = UE[\pmb{Y}] = U\pmb{0} = \pmb{0}
$$

となります．また，$\pmb{Y}$ の分散共分散行列が $\begin{pmatrix} \lambda_1 & 0 \\ 0 & \lambda_2 \end{pmatrix}$ である，すなわち

$$
E\left[(\pmb{Y}-E[\pmb{Y}])(\pmb{Y}-E[\pmb{Y}])^{\top}\right] = E\left[\pmb{Y}\pmb{Y}^{\top}\right] = \begin{pmatrix} \lambda_1 & 0 \\ 0 & \lambda_2 \end{pmatrix}
$$

であることから，$\pmb{X}$ の分散共分散行列は，

$$
\begin{aligned}
E\left[(\pmb{X}-E[\pmb{X}])(\pmb{X}-E[\pmb{X}])^{\top}\right] &= E\left[\pmb{X}\pmb{X}^{\top}\right]\\
&= E\left[ U\pmb{Y}(U\pmb{Y})^{\top} \right]
= E\left[ U\pmb{Y}\pmb{Y}^{\top}U^{\top} \right]\\
&= UE\left[ \pmb{Y}\pmb{Y}^{\top} \right] U^{\top} = U\begin{pmatrix} \lambda_1 & 0 \\ 0 & \lambda_2 \end{pmatrix}U^{\top}\\
&= \Sigma
\end{aligned}
$$

となります．

<br>
<hr width="50%" align="left">
<span style="font-size: 75%">
※注1:分散共分散行列は対称行列なので，その固有ベクトルをならべた行列を用いてこのように対角化できるのでした（ex06notebookC 参照）． </br>
※注2: $E[X]$ は，確率変数 $X$ の期待値を表す記号です．
</span>



これらのことをふまえると，平均 $\pmb{\mu}$ で分散共分散行列が $\Sigma$ の正規乱数は，次のステップの計算で得られることが分かります．

1. $\Sigma$ の固有値 $\lambda_1, \lambda_2$ と，それらに対応する単位固有ベクトル $\pmb{u}_1, \pmb{u}_2$ を求める．固有ベクトルをならべた行列を $U = \begin{pmatrix} \pmb{u}_1 & \pmb{u}_2 \end{pmatrix}$ とする．
1. 平均 $\pmb{0}$ で分散共分散行列が $\begin{pmatrix} \lambda_1 & 0 \\ 0 & \lambda_2 \end{pmatrix}$ の正規乱数を生成する．
1. 得られた乱数値に $U$ を掛けて，平均 $\pmb{0}$ で分散共分散行列が $\Sigma$ の正規乱数を得る．
1. それらに $\pmb{\mu}$ を加えて，平均 $\pmb{\mu}$ で分散共分散行列が $\Sigma$ の正規乱数を得る．

以下のセルでは，$\Sigma =  \begin{pmatrix} 7 & \sqrt{3} \\ \sqrt{3} & 5 \end{pmatrix}$ として $N(\pmb{0}, \Sigma)$ に従う正規乱数を生成します．


In [ ]:
# 平均と分散共分散行列
mu = np.zeros(2)
cov = np.array([[7, np.sqrt(3)], [np.sqrt(3), 5]])

# cov の固有値と固有ベクトルを求める
U, eval, Vt = np.linalg.svd(cov)
U[0, :] *= -1
print(f'固有値 {eval[0]:.1f} 固有ベクトル {U[0, :]}')
print(f'固有値 {eval[1]:.1f} 固有ベクトル {U[1, :]}')

# 2次元標準正規分布に従う乱数値を 500 個取得
Y = rng.standard_normal((500, 2))

# それぞれ分散が eval[0], eval[1] になるように変換
Y[:, 0] *= np.sqrt(eval[0])
Y[:, 1] *= np.sqrt(eval[1])

# X = UY
X = Y @ U

# X の最初の10個を表示
print(X[:10, :])

この場合，固有値は $8, 4$ で，対応する単位固有ベクトルをならべた行列は $U = \begin{pmatrix} \frac{\sqrt{3}}{2} & -\frac{1}{2} \\ \frac{1}{2} & \frac{\sqrt{3}}{2}\end{pmatrix}$ です（注）．したがって，$N\left(\pmb{0}, \begin{pmatrix} 8 & 0 \\ 0 & 4 \end{pmatrix}\right)$ に従う正規乱数（下図左）を作り，それに $U$ をかけて，$N(\pmb{0}, \Sigma)$ に従う正規乱数（下図右）を作っています．


<br>
<hr width="50%" align="left">
<span style="font-size: 75%">
※注: これらを求める問題が ex09notebookC にありました
</span>

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 5))
xmin, xmax = -10, 10
xx, yy = np.mgrid[xmin:xmax:0.01, xmin:xmax:0.01]

# Y の散布図 + PDF
zz = multivariate_normal.pdf(np.dstack((xx, yy)), mean=np.zeros(2), cov=np.diag(eval))
ax[0].scatter(Y[:, 0], Y[:, 1], label='Y', s=8)
ax[0].contour(xx, yy, zz, colors=['#ffa0a0', '#ff5050', '#ff0000'], levels=[0.001, 0.01, 0.1])
ax[0].axvline(0, color='gray')
ax[0].axhline(0, color='gray')
ax[0].set_xlim(-10, 10)
ax[0].set_ylim(-10, 10)
ax[0].set_aspect('equal')
ax[0].legend()

# X の散布図 + PDF
zz = multivariate_normal.pdf(np.dstack((xx, yy)), mean=np.zeros(2), cov=cov)
ax[1].scatter(X[:, 0], X[:, 1], label='X', s=8)
ax[1].contour(xx, yy, zz, colors=['#ffa0a0', '#ff5050', '#ff0000'], levels=[0.001, 0.01, 0.1])
ax[1].axvline(0, color='gray')
ax[1].axhline(0, color='gray')
ax[1].set_xlim(-10, 10)
ax[1].set_ylim(-10, 10)
ax[1].set_aspect('equal')
ax[1].legend()

plt.show()

ここでは自分で $\Sigma$ の固有値や固有ベクトルを求めて変換して，と手間をかけていますが，実際には，NumPy の関数を利用すると簡単にできちゃいます．

In [ ]:
# 平均 mu 分散共分散行列 cov の正規乱数を生成
samples2 = rng.multivariate_normal(mu, cov, size=10)
print(samples2, samples2.shape)

---
### 確率密度（確率分布）の推定

この notebook は「多次元正規分布」に関する資料ですが，このセクションではより広い視点に立って，与えられたデータの背後にある確率分布を推定する（データがどんな確率分布から生み出されたかを推定する）問題を考えます．



次のようなデータ（1000人の身長，単位は[cm]）があります．

In [ ]:
# 1000人の身長
dfHW['height']

「これらのデータは，ある確率分布に従って生成されたものだ」と仮定して，それがどんな確率分布であるかを推定することを考えましょう．例えば，上記のデータが正規分布に従うと仮定して，データからその平均と分散を推定する，ということです（下図参照）．

<img src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/density01.png">

この問題は，モデル（データの背後にある確率分布を定める式，
例では正規分布）を仮定して，データからモデルのパラメータ（例では $\mu$ と $\sigma^2$）を決定する，という話になっています．これは，統計学の分野で古くから研究されてきた問題で，**密度推定**（確率密度の推定）と呼ばれる問題の一種です（注）．

密度推定ができれば，次のようなことが可能となります．
1. データの性質を記述できる．分析や理解が容易になる．
1. 推定された分布を用いて新しいデータを生成できる．

モデルとなる確率分布としては正規分布の他にも様々なものがあり得ますが，以降では，最も典型的なものとして正規分布を取り上げて，この問題についてもう少し考えてみます．

<br>
<hr width="50%" align="left">
<span style="font-size: 75%">
※注: ここでは，モデルとして，正規分布のように少数のパラメータで表されるものを考えています．統計学では，このようなモデルを利用する手法は「パラメトリック(parametric)な手法」と呼ばれています．これに対して，モデルとして特定の確率分布を仮定しない手法もあり，「ノンパラメトリック(non-parametric)な手法」と呼ばれています．
ここで説明しているのは，パラメトリックな密度推定手法です．
</span>


---
### 正規分布のパラメータの推定

#### 1次元正規分布のパラメータ推定の問題設定

母集団の分布が1次元正規分布であると仮定できるときに，その正規分布のパラメータ（平均と分散）を推定する問題を考えます．

**［問題設定］** ある正規分布に従う母集団から $N$ 個の標本 $x_1, x_2, \ldots, x_N$ が抽出された．これらの値から，この母集団の平均と分散を推定したい．ただし，これらの標本は，すべて同一の1次元正規分布に従い，かつ独立に抽出されていると仮定する（注）．

<br>
<hr width="50%" align="left">
<span style="font-size: 75%">
※注: 個々の値が，同一の（平均や分散も同じ）分布から生み出されていると仮定できないと，そもそも平均や分散を一つ決めることができないですね．また，個々の値が独立でない，例えば，1つ目の値に応じて2つ目の値が決まる，というようなことがあってもやはり困ります（後述の最尤推定の話も参照）．このような「独立同一分布」の仮定は統計の問題でよく出てきます．Indenepdent and Identically ditributed の略として，i.i.d. と略記されていることもあります．
</span>

これは，その母集団から抽出した標本（データ）が与えられたときに，それらに最もよく当てはまる1次元正規分布（のパラメータ）を求める問題とも言えます．

In [ ]:
# 身長のヒストグラムと3つの正規分布
fig, ax = plt.subplots(figsize=(10, 5))
xmin, xmax = 140, 200
xx = np.linspace(xmin, xmax, 200)
X = dfHW['height'].to_numpy()
ax.hist(X, density=True, label='height')
mu, sigma = 170, 6
fx = norm.pdf(xx, loc=mu, scale=sigma)
ax.plot(xx, fx, linewidth=3, color='red', label=f'$N({mu},{sigma*sigma})$')
mu, sigma = 160, 7
fx = norm.pdf(xx, loc=mu, scale=sigma)
ax.plot(xx, fx, linewidth=3, color='blue', label=f'$N({mu},{sigma*sigma})$')
mu, sigma = 180, 5
fx = norm.pdf(xx, loc=mu, scale=sigma)
ax.plot(xx, fx, linewidth=3, color='green', label=f'$N({mu},{sigma*sigma})$')
ax.set_xlim(xmin, xmax)
ax.legend()
plt.show()

平均と分散を変化させれば上図のように様々な正規分布が得られますが，このヒストグラムが表す標本に最も良く当てはまるのはどんな平均・分散をもつものだろう，という話です．

後述する「最尤推定」の考え方に基づいて「データに最もよく当てはまる」パラメータの推定値 $\hat{\mu}, \hat{\sigma}^2$ を求めると，次のようになります．
$$
\begin{aligned}
\hat{\mu} &= \frac{1}{N}\sum_{n=1}^{N}x_n & (1)\\
\hat{\sigma}^2 &= \frac{1}{N}\sum_{n=1}^{N}(x_n - \hat{\mu})^2 & (2)
\end{aligned}
$$
これらは，それぞれデータの平均（標本平均）と分散（標本分散）ですね．

この結論は，一見当たり前のことのようにも思えます．ですがこの結論だけでは，「データに最もよく当てはまる」とはどういう意味かがわかりません．
次のセクションでその点を説明をします．

#### 最尤推定

「最尤」は「さいゆう」と読みます．「尤」は，「尤もらしい」＝「もっともらしい」で，「最も」＝「もっとも」と組み合わせて，「最も尤もらしい」ということになります．どういうことかは，以下で...．最尤推定は離散の確率分布でも考えられますが，ここでは連続の場合に限定して説明しています．

ある確率分布の確率密度関数が $f(x)$ であったときに，ある標本値 $x$ がこの確率分布から生成されたと考えることの「尤もらしさ（もっともらしさ）」を表す **尤度**（ゆう度，likelihood）を $\ell(x)$ と表し，次式で定めます．

$$
\ell(x) = f(x)
$$

具体的な例として，下図のように青色（右側）とオレンジ色（左側）の二つの正規分布を考えてみましょう．それぞれの正規分布の平均を $\mu_{\rm blue}, \mu_{\rm orange}$と表し，分散を$\sigma^2_{\rm blue}, \sigma^2_{\rm orange}$ と表すことにします．
あるデータ $x$ が青の正規分布から生成されたとすることの尤度を $\ell_{\rm blue}(x)$ とおき，オレンジの正規分布から生成されたとすることの尤度を $\ell_{\rm orange}(x)$ とおくと，それぞれ次式の通りとなります．
$$
\begin{aligned}
\ell_{\rm blue}(x) &= \frac{1}{\sqrt{2\pi\sigma_{\rm blue}^2}} \exp{\left( - \frac{(x-\mu_{\rm blue})^2}{2\sigma_{\rm blue}^2}\right)} \\
\ell_{\rm orange}(x) &= \frac{1}{\sqrt{2\pi\sigma_{\rm orange}^2}} \exp{\left( - \frac{(x-\mu_{\rm orange})^2}{2\sigma_{\rm orange}^2}\right)} \\
\end{aligned}
$$

<img src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/likelihood.png">

このとき，図の $x_1$ については，$\ell_{\rm blue}(x_1) > \ell_{\rm orange}(x_1)$ が成り立ち（$x_1$は青の正規分布から生成されたとする方が尤もらしい），$x_2$ については，$\ell_{\rm blue}(x_2) < \ell_{\rm orange}(x_2)$ が成り立ちます（$x_2$はオレンジの正規分布から生成されたとする方が尤もらしい）．このように，尤度は，ある決まったパラメータを持つ分布がデータにうまく当てはまる度合いを表します．

標本が $x_1, x_2, \ldots, x_N$ と複数ある場合の尤度は，同時確率を考えて $\ell(x_1, x_2, \ldots, x_N) = f(x_1, x_2, \ldots, x_N) $ と定義されます．独立性の仮定から，これは次式のように個別のデータに対する尤度の積の形になります．
$$
\begin{aligned}
\ell(x_1, x_2, \ldots, x_N) &= f(x_1)\cdot f(x_2)\cdot\cdots\cdot f(x_N) = \prod_{n=1}^{N} f(x_n)
\end{aligned}
$$


上の図のピンクの点と灰色の点が標本値だった場合，$\ell_{\rm orange}(x_1, x_2, \ldots, x_n) >  \ell_{\rm blue}(x_1, x_2, \ldots, x_n) $ となり，これらのデータ全体はオレンジの方の正規分布から生成されたとする方がより尤もらしいと判断できます．

このような考え方に基づいて，尤度を規準としてデータから確率分布のパラメータを推定する方法のことを，**最尤推定**(maximum likelihood estimation)または**最尤法**(maximum likelihood method)といいます．
実は，式$(1)$と$(2)$は，この最尤推定によって得られます．

#### 1次元正規分布のパラメータの最尤推定

以下，1次元正規分布のパラメータを最尤推定によって推定すると，式$(1),(2)$が得られることを示します．
標本全体の尤度は上記のように個々のデータに対する尤度の積で書けます．
これを最大にするようにパラメータを決めればよい，ということになるのですが，積で表した式は扱いにくいので，対数をとったものを考えます．

$$
\begin{aligned}
L &= \log{\ell(x_1, x_2, \ldots, x_N)} = \log\left( \prod_{n=1}^{N} f(x_n) \right) \\
&= \sum_{n=1}^N\log {f(x_n)}
\end{aligned}
$$

これを，**対数尤度**(log-likelihood)といいます．$\log x$ は単調増加関数なので，尤度の最大化は対数尤度の最大化と等価です．

今は正規分布を考えていますので，推定したいパラメータの値を $\mu, \sigma^2$ とおくと，対数尤度 $L$ は次のように変形できます．

$$
\begin{aligned}
L &= \sum_{n=1}^N \log\left( \frac{1}{\sqrt{2\pi\sigma^2}} \exp{\left( - \frac{(x_n-\mu)^2}{2\sigma^2}\right)} \right)\\
&= \sum_{n=1}^N \left( -\frac{1}{2}\log{\left(2\pi\sigma^2\right)} - \frac{(x_n-\mu)^2}{2\sigma^2}\right) \\
&= -\frac{N}{2}\log{2\pi} - \frac{N}{2}\log\sigma^2-\frac{1}{2\sigma^2}\sum_{n=1}^N(x_n-\mu)^2 \qquad (*)
\end{aligned}
$$

$L$を最大にする$\mu,\sigma^2$ を求めたい，ということなので，いつものように微分して $0$ とおく作戦を採ると，式$(1),(2)$が得られます（notebookB の方でその確認を行うことにします）．


身長のデータに1次元正規分布を当てはめて平均と分散を最尤推定してみると，次のようになります．

In [ ]:
# 標本
X = dfHW['height'].to_numpy()

# 平均と分散を推定
mu = np.mean(X)
sig2 = np.var(X)

# データのヒストグラムと推定された正規分布を描く
xx = np.linspace(140, 200, num=100)
px = norm.pdf(xx, loc=mu, scale=np.sqrt(sig2))
fig, ax = plt.subplots(1, facecolor="white", figsize=(6, 4))
ax.hist(X, bins=20, density=True)
ax.plot(xx, px, linewidth=3, color='red')
ax.set_xlim(140, 200)
plt.show()
print(f'平均: {mu:.2f}  分散: {sig2:.2f}')

#### 多次元正規分布のパラメータ推定

多次元正規分布の場合にも，1次元の場合と同様にパラメータ（平均と分散共分散行列）の最尤推定を考えることができます．

多次元正規分布に従う母集団から抽出された $N$ 個の標本を $\pmb{x}_1, \pmb{x}_2, \ldots, \pmb{x}_N$ （いずれも$D\times 1$行列とする）とするとき，この正規分布の平均と分散共分散行列の最尤推定量 $\hat{\pmb{\mu}}$ と $\hat{\Sigma}$ は次式のように与えられます（導出は省略します）．

$$
\begin{aligned}
\hat{\pmb{\mu}} &= \frac{1}{N}\sum_{n=1}^{N}\pmb{x}_n & (3)\\
\hat{\Sigma} &= \frac{1}{N}\sum_{n=1}^{N}(\pmb{x}_n-\hat{\pmb{\mu}})(\pmb{x}_n-\hat{\pmb{\mu}})^{\top} & (4)
\end{aligned}
$$

それぞれ，データの標本平均と分散共分散行列そのものに一致しています．



(身長, 体重)のデータに2次元の正規分布を当てはめる実験をやってみましょう．

In [ ]:
dfHW

In [ ]:
# 標本
X = dfHW.to_numpy()
N, D = X.shape

# 平均と分散共分散行列を推定
mu = np.mean(X, axis=0)
cov = (X - mu).T @ (X - mu)/N

# グラフ描画用のグリッドデータの作成
xmin, xmax = 130, 210
ymin, ymax = 20, 100
xx, yy = np.mgrid[xmin:xmax:1, ymin:ymax:1]
zz = multivariate_normal.pdf(np.dstack((xx, yy)), mu ,cov)

# データの散布図と推定された正規分布の等高線を描く
fig, ax = plt.subplots(facecolor='white', figsize=(6, 6))
ax.scatter(X[:, 0], X[:, 1], s=8)
ax.contour(xx, yy, zz, colors=['#ffa0a0', '#ff5050', '#ff0000'], levels=[0.00001, 0.0001, 0.001])
ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)
ax.set_xlabel('height')
ax.set_ylabel('weight')
ax.set_aspect('equal')
plt.show()
print(f'N = {N}  平均: {mu}')
print(f'分散共分散行列:\n {cov}')

#### 最尤推定量は不偏推定量とは限らない

ここで登場する「不偏推定量」という概念は，大学初年次の確率統計の授業で学んでいると思います．詳しくはそちらの教科書や資料等を参照してください．

1次元正規分布および多次元正規分布のパラメータを標本値から最尤推定した際に得られる推定量は式$(1),(2),(3),(4)$で表されます．これらは，尤度を最大にするという意味で適切な推定量になっていますが，「不遍性」（推定量の期待値が真の値に等しくなるという性質）の点では適切でないものもあります．

式$(1),(3)$は「標本平均」であり，これらは母集団の平均の不偏推定量であることが知られています．一方，式$(2)$は「標本分散」と呼ばれるものですが，これは母集団の分散の不偏推定量ではありません（不遍性がない）．$N$個の標本から母集団の分散を推定する際の不偏推定量は，次式で定められる「不偏分散」です．

$$
\tilde{\sigma}^2 = \frac{1}{N-1}\sum_{n=1}^{N}(x_n - \hat{\mu})^2
$$

同様に，式$(4)$で得られる「標本分散共分散行列」も不遍性がありません．こちらも，$N$の代わりに$N-1$で割ったものが不偏推定量となります．